In [ ]:
import pandas as pd
dataset=pd.read_csv('../input/Kannada-MNIST/train.csv')
dataset.head()

In [ ]:
dataset.info()

In [ ]:
dataset.nunique().sort_values(ascending=False)[:5]

In [ ]:
data = dataset.sample(frac=0.8, random_state=42)
data_unseen = dataset.drop(data.index)
data.reset_index(inplace=True, drop=True)
data_unseen.reset_index(inplace=True, drop=True)
print('Data for Modeling: ' + str(data.shape))

In [ ]:
print('Unseen Data For Predictions: ' + str(data_unseen.shape))

In [ ]:
!pip install pycaret

In [ ]:
from pycaret.classification import *

In [ ]:
numeric_features=data.columns.tolist()
numeric_features.remove('label')
numeric_features[:5]

In [ ]:
data.label.value_counts()

In [ ]:
exp_clf101 = setup(data = data, target = 'label', session_id=42,
                  normalize = True, 
                  transformation = True, 
                  log_experiment = True,
                  handle_unknown_categorical = True, 
                  unknown_categorical_method = 'most_frequent',
                  remove_multicollinearity = True, #rop one of the two features that are highly correlated with each other
                  ignore_low_variance = True,#all categorical features with statistically insignificant variances are removed from the dataset.
                  combine_rare_levels = True,# all levels in categorical features below the threshold defined in rare_level_threshold param are combined together as a single level
                  numeric_imputation='median',
                  categorical_imputation='mode',
                   numeric_features=numeric_features)

In [ ]:
lightgbm = create_model('lightgbm')

In [ ]:
print(lightgbm)

In [ ]:
tuned_lightgbm = tune_model(lightgbm,optimize = 'Precision')

In [ ]:
plot_model(tuned_lightgbm, plot = 'auc')

In [ ]:
plot_model(tuned_lightgbm, plot = 'pr')

In [ ]:
plot_model(tuned_lightgbm, plot='feature')

In [ ]:
plot_model(tuned_lightgbm, plot = 'confusion_matrix')

In [ ]:
evaluate_model(tuned_lightgbm)

In [ ]:
predict_model(tuned_lightgbm)

In [ ]:
unseen_predictions = predict_model(tuned_lightgbm, data=data_unseen)
unseen_predictions.head()

In [ ]:
print("Confidence Score :   {}".format(round(unseen_predictions.Score.mean(),2)))#Confidence Score

In [ ]:
test=pd.read_csv('../input/Kannada-MNIST/test.csv')
test.head()

In [ ]:
sol = predict_model(tuned_lightgbm, data=test)
sol.head()

In [ ]:
final=pd.read_csv('../input/Kannada-MNIST/sample_submission.csv')
final.head()

In [ ]:
final.Label=sol.Label
final.head()

In [ ]:
final.to_csv('final.csv', index=False)